###### ### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2021 Semester 1

## Assignment 1: Pose classification with naive Bayes


**Student ID(s):**     `PLEASE ENTER YOUR ID(S) HERE`


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [109]:
# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing

import pandas as pd
import numpy as np
from collections import defaultdict
import re
from math import sqrt
from math import exp
from math import pow
from math import pi
from math import log
import matplotlib.pyplot as plt
%matplotlib inline

def preprocess(columns):
    
    filename = "COMP30027_2021_assignment1_data/train.csv"
    f = open(filename,'r')
    
    # impute missing values of 9999 to NaN
    train_collection = pd.read_csv(filename,names = columns,na_values=[9999]) 
    
    # number of instances of each class
    train_instance_count = train_collection.groupby(["class_name"]).size().reset_index(name='size').set_index("class_name")
    
    class_names = train_collection.class_name.unique()
    
    f.close
    
    return train_collection, train_instance_count, class_names
    

In [287]:
# This function should calculate prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train(training_df,train_instance_count):
    
    # standard deviations of the attributes of each class
    standard_devs = training_df.groupby(["class_name"]).std()
    
    # mean of the attributes of each class
    means = training_df.groupby(["class_name"]).mean()
    
    # class probabilities of each class
    class_priors = pd.DataFrame(train_instance_count['size'].div(float(len(training_df))))

    return standard_devs, means, class_priors


In [298]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def gaussian_distribution(val,mean,std_dev):
    exponential = exp(-(1/2)* (pow(((val-mean)/std_dev),2)))
    result = (1/(std_dev * sqrt(2*pi)))* exponential
    return result

#log to handle NaN and 0
def take_log(val):
    if(np.isnan(val) or val == 0):
        return 0.0
    else:
        return log(val)

# find the probability of the instance being classified as each class
def probability(instance,means,std_dev,attr_num,class_priors,class_names):
    
    probs = defaultdict()
    attributes = means.columns

    for key in class_names:
        total_probs = 0
        sum_gaussian_probs = 0
        
        class_means = means.loc[key]
        class_std_devs = std_dev.loc[key]
        
        for one_att in attributes:
            #mean of that attribute
            mean = class_means.loc[one_att]
            
            #standard deviation of that attribute
            standard_dev = class_std_devs.loc[one_att]
            
            #find the gaussian probability of that attribute
            gauss_prob = gaussian_distribution(instance.loc[one_att],mean,standard_dev)
            
            #sum(log(each attribute)
            sum_gaussian_probs += take_log(gauss_prob)
            
        # probability of the instance being a class :  log(class prob) + sum(log(each attribute))
        total_probs += sum_gaussian_probs
        class_prob = class_priors.loc[key]
        total_probs += log(class_prob)
        
        # put in dictionary
        probs[key] = total_probs

    return probs
            
def predict(means,std_dev,class_priors,class_names):
    
    filename = "COMP30027_2021_assignment1_data/test.csv"
    f = open(filename,'r')
    
    # impute missing values with NaN
    test_collection = pd.read_csv(filename, names = columns,na_values=[9999]) 

    attribute_num = len(test_collection.columns)-1
    test_instance_num = len(test_collection)
    predicted_class = []

    for i in range(test_instance_num):
        
        # don't take the instance's actual class_name
        instance = test_collection.iloc[i,1:]
        
        # dictionary contains the probability of the instance being in each classes 
        probability_dict = probability(instance,means,std_dev,attribute_num,class_priors,class_names)
        
        # find the class name with the highest probability
        class_max_prob = max(probability_dict, key = probability_dict.get)
        
        # store the class name with the highest probability
        predicted_class.append(class_max_prob)
   
    # add the predicted classes to the dataframe
    test_collection['predicted_class'] = predicted_class  
    
    return test_collection

    

In [326]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate(test_collection):
    same = 0
    instance_num = len(test_collection)
    actual_class = test_collection['class_name']
    predicted_class = test_collection['predicted_class']
    
    for i in range(instance_num):
        if actual_class.at[i]==predicted_class.at[i]:
            same+=1
            
    accuracy_percentage = (float(same)/float(instance_num))
    
    return accuracy_percentage

In [328]:
#RUN HERE
columns = ["class_name","x1","x2","x3","x4","x5","x6","x7","x8","x9","x10","x11","y1","y2","y3","y4","y5","y6","y7","y8","y9","y10","y11"]
training_df = pd.DataFrame()
train_instance_count = pd.DataFrame()
class_names=[]

training_df,train_instance_count,class_names = preprocess(columns)
standard_devs, means, class_priors = train(training_df,train_instance_count)

test_collection = predict(means,standard_devs,class_priors,class_names)

accuracy = evaluate(test_collection)

print("Accuracy is:",accuracy)

Accuracy is: 0.7155172413793104


## Questions 


If you are in a group of 1, you will respond to **two** questions of your choosing.

If you are in a group of 2, you will respond to **four** questions of your choosing.

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions to help respond to the question here, but your formal answer should be submitted separately as a PDF.

### Q1
Since this is a multiclass classification problem, there are multiple ways to compute precision, recall, and F-score for this classifier. Implement at least two of the methods from the "Model Evaluation" lecture and discuss any differences between them. (The implementation should be your own and should not just call a pre-existing function.)

### Q2
The Gaussian naıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in this dataset? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the classifier’s predictions.

### Q3
Implement a kernel density estimate (KDE) naive Bayes classifier and compare its performance to the Gaussian naive Bayes classifier. Recall that KDE has kernel bandwidth as a free parameter -- you can choose an arbitrary value for this, but a value in the range 5-25 is recommended. Discuss any differences you observe between the Gaussian and KDE naive Bayes classifiers. (As with the Gaussian naive Bayes, this KDE naive Bayes implementation should be your own and should not just call a pre-existing function.)

### Q4
Instead of using an arbitrary kernel bandwidth for the KDE naive Bayes classifier, use random hold-out or cross-validation to choose the kernel bandwidth. Discuss how this changes the model performance compared to using an arbitrary kernel bandwidth.

### Q5
Naive Bayes ignores missing values, but in pose recognition tasks the missing values can be informative. Missing values indicate that some part of the body was obscured and sometimes this is relevant to the pose (e.g., holding one hand behind the back). Are missing values useful for this task? Implement a method that incorporates information about missing values and demonstrate whether it changes the classification results.

### Q6
Engineer your own pose features from the provided keypoints. Instead of using the (x,y) positions of keypoints, you might consider the angles of the limbs or body, or the distances between pairs of keypoints. How does a naive Bayes classifier based on your engineered features compare to the classifier using (x,y) values? Please note that we are interested in explainable features for pose recognition, so simply putting the (x,y) values in a neural network or similar to get an arbitrary embedding will not receive full credit for this question. You should be able to explain the rationale behind your proposed features. Also, don't forget the conditional independence assumption of naive Bayes when proposing new features -- a large set of highly-correlated features may not work well.